In [1]:
#PURE PYTHON!!!!
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os
import re

In [66]:
path = %pwd
path += '/fifa-18-demo-player-dataset/CompleteDataset.csv'

In [67]:
def clean_raw_data(path=None):
    """
    Denne metode fjerner uønskede kolonner samt indsætter 0 på målmændendes ikke-målmænd attributter
    @input: path: sti til hvor data ligger
    """
    raw_data = (pd
                .DataFrame
                .from_csv(
                    path=path,
                    encoding='utf-8',
                    index_col='ID'
                ))
    del raw_data['Club Logo']
    del raw_data['Photo']
    del raw_data['Flag']
    del raw_data['Unnamed: 0']
    raw_data = convert_object_col_to_float(
        raw_data.fillna(0))
    return raw_data
    

In [68]:
def convert_object_col_to_float(pdf):
    """
    
    """
    
    remove_funny_stuff = lambda x: float(re.sub('\W\d+','',x))
    for col in pdf.columns[9:43]: # Vi snyder, for vi ved godt at fejlene ligger i disse kolonner...
        try:
            if pdf.dtypes[col] == np.object:
                pdf[col] = pdf[col].map(str)
                pdf[col] = pdf[col].map(remove_funny_stuff)
        except Exception as e:
            print(col+" "+str(e))
            break
    return pdf

In [85]:
df = clean_raw_data(path)

/home/sidsel/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2847: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [70]:
df

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Acceleration,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
ID,,,,,,,,,,,,,,,,,,,,,
20801,Cristiano Ronaldo,32,Portugal,94,94,Real Madrid CF,€95.5M,€565K,2228,89.0,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
158023,L. Messi,30,Argentina,93,93,FC Barcelona,€105M,€565K,2154,92.0,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
190871,Neymar,25,Brazil,92,94,Paris Saint-Germain,€123M,€280K,2100,94.0,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
176580,L. Suárez,30,Uruguay,92,92,FC Barcelona,€97M,€510K,2291,88.0,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
167495,M. Neuer,31,Germany,92,92,FC Bayern Munich,€61M,€230K,1493,58.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188545,R. Lewandowski,28,Poland,91,91,FC Bayern Munich,€92M,€355K,2143,79.0,...,58.0,57.0,78.0,62.0,87.0,82.0,88.0,84.0,61.0,88.0
193080,De Gea,26,Spain,90,92,Manchester United,€64.5M,€215K,1458,57.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183277,E. Hazard,26,Belgium,90,91,Chelsea,€90.5M,€295K,2096,93.0,...,59.0,47.0,81.0,61.0,87.0,87.0,82.0,88.0,64.0,82.0
182521,T. Kroos,27,Germany,90,90,Real Madrid CF,€79M,€340K,2165,60.0,...,76.0,72.0,87.0,82.0,81.0,81.0,77.0,80.0,78.0,77.0


In [71]:
df['Value'] = df['Value'].map(lambda x: re.sub('€','',x))
df['Wage'] = df['Wage'].map(lambda x: re.sub('€','',x))


In [72]:
c = df['Value'].map(lambda x: re.findall('(\d+\.?\d)([MK])',x))
s = c[c.index == 20801]
float(s.tolist()[0][0][0])

95.5

In [113]:
desired_clubs = ['Real Madrid CF', 'FC Barcelona', 'FC Bayern Munich', 'Paris Saint-Germain', 'Chelsea', 'Manchester City']

In [103]:
def parse_of_wage(val):
    val = re.sub('€', '', val)
    valdict = {'M': 1000000, 'K': 1000, '': 0}
    
    try:
        splitter = re.findall('(\d*\.?\d)([MK]?)', val)[0]
        return float(splitter[0])*valdict[splitter[1]]
    except IndexError as e:
        print(splitter)

In [106]:
df['Value'] = df['Value'].map(lambda x: parse_of_wage(x))
df['Wage'] = df['Wage'].map(lambda x: parse_of_wage(x))


### Classification 

In [134]:
df['label'] = df['Club'].isin(desired_clubs).astype(int)

In [146]:
test_dansker = df[df['Nationality'] == 'Denmark'] 
test_dansker

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Acceleration,...,RCM,RDM,RF,RM,RS,RW,RWB,ST,temp,label
ID,,,,,,,,,,,,,,,,,,,,,
190460,C. Eriksen,25,Denmark,87,91,Tottenham Hotspur,65000000.0,165000.0,2064,77.0,...,83.0,68.0,82.0,84.0,77.0,83.0,69.0,77.0,65000000.0,0
163587,K. Schmeichel,30,Denmark,83,84,Leicester City,19000000.0,65000.0,1355,62.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19000000.0,0
213661,A. Christensen,21,Denmark,81,89,Chelsea,23000000.0,86000.0,1720,70.0,...,67.0,77.0,58.0,64.0,55.0,60.0,74.0,55.0,23000000.0,1
179645,S. Kjær,28,Denmark,81,82,Sevilla FC,16000000.0,23000.0,1869,60.0,...,65.0,76.0,57.0,59.0,61.0,56.0,72.0,61.0,16000000.0,0
172522,D. Wass,28,Denmark,80,80,RC Celta de Vigo,14500000.0,33000.0,2107,73.0,...,79.0,74.0,78.0,79.0,75.0,77.0,76.0,75.0,14500000.0,0
202849,J. Vestergaard,24,Denmark,79,84,Borussia Mönchengladbach,14500000.0,36000.0,1712,47.0,...,63.0,74.0,54.0,54.0,58.0,50.0,66.0,58.0,14500000.0,0
199852,N. Jørgensen,26,Denmark,79,81,Feyenoord,15500000.0,25000.0,1931,76.0,...,71.0,57.0,78.0,76.0,78.0,76.0,59.0,78.0,15500000.0,0
159240,M. Krohn-Dehli,34,Denmark,79,79,Sevilla FC,6500000.0,24000.0,2084,69.0,...,79.0,74.0,78.0,78.0,74.0,78.0,73.0,74.0,6500000.0,0
228687,K. Dolberg,19,Denmark,78,88,Ajax,16500000.0,10000.0,1834,73.0,...,66.0,49.0,76.0,70.0,77.0,72.0,50.0,77.0,16500000.0,0


In [140]:
df_train = df.loc[df['Nationality'] != 'Denmark']

In [141]:
df_train

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Acceleration,...,RCM,RDM,RF,RM,RS,RW,RWB,ST,temp,label
ID,,,,,,,,,,,,,,,,,,,,,
20801,Cristiano Ronaldo,32,Portugal,94,94,Real Madrid CF,95500000.0,565000.0,2228,89.0,...,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0,95500000.0,1
158023,L. Messi,30,Argentina,93,93,FC Barcelona,105000000.0,565000.0,2154,92.0,...,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0,105000000.0,1
190871,Neymar,25,Brazil,92,94,Paris Saint-Germain,123000000.0,280000.0,2100,94.0,...,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0,123000000.0,1
176580,L. Suárez,30,Uruguay,92,92,FC Barcelona,97000000.0,510000.0,2291,88.0,...,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0,97000000.0,1
167495,M. Neuer,31,Germany,92,92,FC Bayern Munich,61000000.0,230000.0,1493,58.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61000000.0,1
188545,R. Lewandowski,28,Poland,91,91,FC Bayern Munich,92000000.0,355000.0,2143,79.0,...,78.0,62.0,87.0,82.0,88.0,84.0,61.0,88.0,92000000.0,1
193080,De Gea,26,Spain,90,92,Manchester United,64500000.0,215000.0,1458,57.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64500000.0,0
183277,E. Hazard,26,Belgium,90,91,Chelsea,90500000.0,295000.0,2096,93.0,...,81.0,61.0,87.0,87.0,82.0,88.0,64.0,82.0,90500000.0,1
182521,T. Kroos,27,Germany,90,90,Real Madrid CF,79000000.0,340000.0,2165,60.0,...,87.0,82.0,81.0,81.0,77.0,80.0,78.0,77.0,79000000.0,1


In [147]:
assert len(df) - len(test_dansker) == len(df_train) , 'We didn\'t get all the danes'

In [180]:
X_train = pd.concat([
    df_train.loc[df_train['label']==0].sample(170),
    df_train.loc[df_train['label']==1]
]).select_dtypes(include=['float64', 'int'])

In [183]:
y_train = X_train['label']

In [181]:
X_train

,Age,Overall,Potential,Value,Wage,Special,Acceleration,Aggression,Agility,Balance,...,RCM,RDM,RF,RM,RS,RW,RWB,ST,temp,label
ID,,,,,,,,,,,,,,,,,,,,,
212712,26,65,67,525000.0,2000.0,1658,64.0,70.0,68.0,61.0,...,58.0,62.0,57.0,60.0,56.0,58.0,64.0,56.0,525000.0,0
234618,25,74,75,6500000.0,10000.0,1761,77.0,58.0,71.0,68.0,...,67.0,52.0,73.0,74.0,70.0,74.0,56.0,70.0,6500000.0,0
233575,24,63,69,425000.0,2000.0,1562,67.0,62.0,64.0,63.0,...,56.0,60.0,51.0,58.0,48.0,55.0,63.0,48.0,425000.0,0
207441,23,79,86,18000000.0,60000.0,1979,76.0,66.0,78.0,88.0,...,72.0,57.0,80.0,78.0,78.0,80.0,59.0,78.0,18000000.0,0
201495,28,64,64,475000.0,5000.0,1561,76.0,48.0,69.0,71.0,...,50.0,39.0,61.0,59.0,63.0,61.0,43.0,63.0,475000.0,0
216778,23,78,85,12500000.0,27000.0,1806,68.0,87.0,66.0,51.0,...,63.0,72.0,60.0,60.0,62.0,59.0,69.0,62.0,12500000.0,0
208508,29,68,68,900000.0,19000.0,1558,58.0,66.0,51.0,46.0,...,53.0,42.0,64.0,58.0,67.0,60.0,42.0,67.0,900000.0,0
209256,23,64,69,500000.0,11000.0,1335,37.0,83.0,33.0,44.0,...,43.0,56.0,36.0,38.0,41.0,34.0,49.0,41.0,500000.0,0
200981,25,65,65,600000.0,4000.0,1618,75.0,49.0,71.0,58.0,...,56.0,46.0,64.0,63.0,65.0,65.0,51.0,65.0,600000.0,0


# Logistic regression model

In [148]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

In [150]:
classifier = LogisticRegression(random_state=0)


In [167]:
model = classifier.fit(X_train, y_train)

In [169]:
model.score(X_train, y_train)

0.99228806351006527

ID
20801     1
158023    1
190871    1
176580    1
167495    1
188545    1
193080    0
183277    1
182521    1
167664    0
155862    1
192985    1
192119    1
184941    0
177003    1
173731    1
153079    1
138956    0
1179      0
211110    0
200389    0
194765    0
189509    1
188567    0
184344    0
183907    1
182493    0
178603    1
176635    0
167948    0
         ..
221669    0
239079    0
238833    0
225778    0
231928    0
240381    0
236807    0
237859    0
225319    0
236597    0
233047    1
236425    0
238217    0
238985    0
238219    0
238477    0
231824    0
237974    0
236954    0
231107    0
240325    0
238306    0
240403    0
240404    0
235352    0
237463    0
11728     0
231381    0
238813    0
238308    0
Name: label, dtype: int64

In [171]:
df_train.groupby('label').count()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Acceleration,...,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST,temp
label,,,,,,,,,,,,,,,,,,,,,
0,17465,17465,17465,17465,17465,17465,17465,17465,17465,17465,...,17465,17465,17465,17465,17465,17465,17465,17465,17465,17465
1,170,170,170,170,170,170,170,170,170,170,...,170,170,170,170,170,170,170,170,170,170
